No interactions <br>
Simple Imputes new category for categorical and mode for numerical <br>
MinMaxScaler

In [54]:
import pathlib
import os
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegressionCV
from sklearn.impute import SimpleImputer
from sklearn import set_config
import joblib

# Enable pandas output globally
set_config(transform_output="pandas")

In [3]:
# Get path to train file
train_path = os.path.join(pathlib.Path.cwd().parent, "raw", "train.csv")
test_path = os.path.join(pathlib.Path.cwd().parent, "raw", "test.csv")
train_path

'c:\\Users\\bogus\\Desktop\\Projects\\Kaggle competitions\\Introverts\\raw\\train.csv'

In [4]:
train_df = pd.read_csv(train_path, index_col=0)
test_df = pd.read_csv(test_path, index_col=0)
train_df.head()

,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
id,,,,,,,,
0,0.0,No,6.0,4.0,No,15.0,5.0,Extrovert
1,1.0,No,7.0,3.0,No,10.0,8.0,Extrovert
2,6.0,Yes,1.0,0.0,NaN,3.0,0.0,Introvert
3,3.0,No,7.0,3.0,No,11.0,5.0,Extrovert
4,1.0,No,4.0,4.0,No,13.0,NaN,Extrovert


In [8]:
numerical_vars = train_df.select_dtypes(include='number').columns.tolist()
categorical_vars = train_df.drop(labels="Personality", axis=1).select_dtypes(include='object').columns.tolist()
y_var = "Personality"
numerical_vars, categorical_vars, y_var

(['Time_spent_Alone',
  'Social_event_attendance',
  'Going_outside',
  'Friends_circle_size',
  'Post_frequency'],
 ['Stage_fear', 'Drained_after_socializing'],
 'Personality')

## Pipeline

In [48]:
numerical_pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("minmax", MinMaxScaler())
    ]
)

categorical_pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="constant", fill_value="Missing")),
        ("onehot", OneHotEncoder(drop="if_binary", sparse_output=False))
    ]
)

preprocessing_pipeline = Pipeline(
    [
        ("column_transformer", ColumnTransformer(
            [
                ("numerical", numerical_pipeline, numerical_vars),
                ("categorical", categorical_pipeline, categorical_vars)
            ]
        ))
    ]
)

pipeline = Pipeline(
    [
        ("preprocessing", preprocessing_pipeline),
        ("logistic_regression", LogisticRegressionCV(fit_intercept=True, cv=5))
    ]
)

In [49]:
pipeline.fit(train_df, train_df[y_var])

Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('column_transformer',
                                  ColumnTransformer(transformers=[('numerical',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer(strategy='most_frequent')),
                                                                                   ('minmax',
                                                                                    MinMaxScaler())]),
                                                                   ['Time_spent_Alone',
                                                                    'Social_event_attendance',
                                                                    'Going_outside',
                                                                    'Friends_circle_size',
                                                                    'Post_frequency']),
                                                                  ('categorical',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer(fill_value='Missing',
                                                                                                  strategy='constant')),
                                                                                   ('onehot',
                                                                                    OneHotEncoder(drop='if_binary',
                                                                                                  sparse_output=False))]),
                                                                   ['Stage_fear',
                                                                    'Drained_after_socializing'])]))])),
                ('logistic_regression', LogisticRegressionCV(cv=5))])

In [50]:
pipeline.score(train_df, train_df[y_var])

0.9690131721010581

## Predict on test data and create submission file

In [51]:
test_predict = pipeline.predict(test_df)
test_predict

array(['Extrovert', 'Introvert', 'Extrovert', ..., 'Extrovert',
       'Extrovert', 'Introvert'], shape=(6175,), dtype=object)

In [52]:
test_df = pd.DataFrame(
    {
        "id": test_df.index,
        y_var: test_predict
    }
)

In [53]:
test_df.to_csv("submission.csv", index=False)

## save model

In [55]:
joblib.dump(pipeline, "pipeline.pkl")

['pipeline.pkl']